In [ ]:
%matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [ ]:
path='/home/ori/projects/data2/190227-121527/'
#imname='r00007935.pgm'
#imname='r00003575.pgm'
imname='r00003480.pgm'
#imname='r00009175.pgm'
imname='l00006440.pgm'
imname='r00006440.pgm'
fname=path+imname
fname='/home/ori/projects/RovVision2/plugins/tracker/iml.png'

In [ ]:
im = cv2.imread(fname)[::2,::2,1]
plt.figure()
plt.imshow(im,cmap='gray')

In [ ]:
slot=[100,200]
im1=im[slot[0]:slot[1],:].sum(axis=0).flatten().astype(float)
plt.figure()
#plt.plot(np.convolve(im1,np.concatenate((np.ones(8),-np.ones(8)))))
im2=im1#np.convolve(im1,np.ones(10),'valid')

plt.plot(im2)


In [ ]:
yy=np.fft.fft(im2)
y=np.log10(np.abs(yy))
x=np.fft.fftfreq(len(im2))
plt.figure()
#plt.plot(x,y)
yy[5:-5]=0
im2r=np.fft.ifft(yy)
plt.plot(im2r)

In [ ]:
from scipy.signal import argrelextrema
argrelextrema(im2r, np.greater)[0]

In [ ]:
argrelextrema(im2r, np.less)

In [ ]:
def detect_rope(prev_col,extrema,start_row,nrows,im):
    imt=im[start_row:start_row+nrows,:].sum(axis=0).flatten().astype(float)
    #plt.plot(np.convolve(im1,np.concatenate((np.ones(8),-np.ones(8)))))
    #conv_val=10
    #im2=np.convolve(im1,np.ones(conv_val),'valid')
    yy=np.fft.fft(imt)
    yy[5:-5]=0
    im2r=np.fft.ifft(yy)
    
    maxima=argrelextrema(im2r, np.greater)[0]
    minima=argrelextrema(im2r, np.less)[0]
    if extrema is None:
        totest=np.concatenate((maxima,minima))
    elif extrema == 'max':
        totest=maxima
    else:
        totest=minima
    if len(totest)==0:
        return None
    new_col=totest[np.argmin(np.abs(totest-prev_col))]
    res='min'
    if new_col in maxima:
        res='max'
    return res,new_col
        

In [ ]:

detect_rope(329,None,100,100,im)

In [ ]:
thick=200
extrema,col=detect_rope(100,None,0,thick,im)
cols=[]
for i in range(im.shape[0]-thick):
    extrema,col=detect_rope(col,extrema,i,100,im)
    cols.append((col,i+thick//2))
cols=np.array(cols)

In [ ]:
plt.figure()
plt.imshow(im,cmap='gray')
plt.plot(cols[:,0],cols[:,1],'-r')